In [2]:
import psycopg2

In [3]:
connect = psycopg2.connect(dbname="fnf", host="prd-dt-redshift.conhugwtudej.ap-northeast-2.redshift.amazonaws.com", port=5439, user="wecode", password="Wecode@2022!#")

In [4]:
cursor = connect.cursor()

In [5]:
brand = 'M'
season = "'22S'"
season_py = "'21S'"
season_py2 = "'20S'"
category = '맨투맨'
sub_category = "'후드','맨투맨'"
adult_kids = '성인'
start_dt = '2021-12-28'
end_dt = '2022-02-06'
end_dt_this_week = '2022-02-06'

In [6]:
query = """

with sale as (
    select '당해'                                         term_cls
         , ac_stor_qty_kor                           as ac_stor_qty_kor
         , sale_nml_qty_cns + sale_ret_qty_cns       as sale_qty_kor
         , ac_sale_nml_qty_cns + ac_sale_ret_qty_cns as ac_sale_qty_kor
         , stock_qty                                 as stock_qty
    from prcs.db_scs_w a,
         prcs.db_prdt b
    where a.prdt_cd = b.prdt_cd
      and a.brd_cd = '{para_brand}'
      and a.sesn in ({para_season}) --당해
      and cat_nm = '{para_category}'
      and sub_cat_nm in ({para_sub_category})
      and adult_kids_nm = '{para_adult_kids}'
      and end_dt = '{para_end_dt_this_week}'
    union all
    select '전년'                                         term_cls
         , ac_stor_qty_kor                           as ac_stor_qty_kor
         , sale_nml_qty_cns + sale_ret_qty_cns       as sale_qty_kor
         , ac_sale_nml_qty_cns + ac_sale_ret_qty_cns as ac_sale_qty_kor
         , stock_qty                                 as stock_qty
    from prcs.db_scs_w a,
         prcs.db_prdt b
    where a.prdt_cd = b.prdt_cd
      and a.brd_cd = '{para_brand}'
      and a.sesn in ({para_season_py}) --전년
      and cat_nm = '{para_category}'
      and sub_cat_nm in ({para_sub_category})
      and adult_kids_nm = '{para_adult_kids}'
      and end_dt = '{para_end_dt_this_week}' - 364
),
     order_status as (
         select '당해'  as term_cls
              , b.qty as indc_qty
         from prcs.dw_ord a,
              prcs.dw_ord_scs b,
              prcs.db_prdt c
         where a.prdt_cd = b.prdt_cd
           and a.po_no = b.po_no
           and a.prdt_cd = c.prdt_cd
           and a.brd_cd = '{para_brand}'
           and a.sesn in ({para_season}) --당해
           and cat_nm = '{para_category}'
           and sub_cat_nm in ({para_sub_category})
           and adult_kids_nm = '{para_adult_kids}'
           and apv_stat = 'C'
           and po_cntry in ('A', 'KR')
           and po_cust_cntry != 'M'
           and apv_dt <= '{para_end_dt_this_week}'
         union all
         select '전년'  as term_cls
              , b.qty as indc_qty
         from prcs.dw_ord a,
              prcs.dw_ord_scs b,
              prcs.db_prdt c
         where a.prdt_cd = b.prdt_cd
           and a.po_no = b.po_no
           and a.prdt_cd = c.prdt_cd
           and a.brd_cd = '{para_brand}'
           and a.sesn in ({para_season_py}) --전년
           and cat_nm = '{para_category}'
           and sub_cat_nm in ({para_sub_category})
           and adult_kids_nm = '{para_adult_kids}'
           and po_cntry in ('A', 'KR')
           and po_cust_cntry != 'M'
           and apv_stat = 'C'
           and apv_dt <= '{para_end_dt_this_week}' - 364
     ),
     season_end as (
         select '전년'                                         term_cls
              , ac_stor_qty_kor                           as ac_stor_qty_kor_season_end
              , ac_sale_nml_qty_cns + ac_sale_ret_qty_cns as ac_sale_qty_kor_season_end
              , stock_qty                                 as stock_qty_season_end
         from prcs.db_scs_w a,
              prcs.db_prdt b
         where a.prdt_cd = b.prdt_cd
           and a.brd_cd = '{para_brand}'
           and a.sesn in ({para_season_py}) --전년
           and cat_nm = '{para_category}'
           and sub_cat_nm in ({para_sub_category})
           and adult_kids_nm = '{para_adult_kids}'
           and (
             select max(last_week)
             from (
                      select case
                                 when substring(a.sesn, 3, 1) = 'S'
                                     then date(date_trunc('week', date('20' || substring(a.sesn, 1, 2) || '0901'))) + 6
                                 when substring(a.sesn, 3, 1) = 'F'
                                     then
                                     date(date_trunc('week', date('20' || substring(a.sesn, 1, 2)::integer + 1 || '0301'))) + 6
                                 else
                                     date(date_trunc('week', date('20' || substring(a.sesn, 1, 2) || '1231'))) + 6
                                 end as last_week
                      from prcs.db_scs_w a,
                           prcs.db_prdt b
                      where a.brd_cd = b.brd_cd
                        and a.prdt_cd = b.prdt_cd
                        and a.brd_cd = '{para_brand}'
                        and cat_nm = '{para_category}'
                        and adult_kids_nm = '{para_adult_kids}'
                        and sub_cat_nm in ({para_sub_category})
                        and a.sesn in ({para_season_py})
                  ) a
         ) between start_dt and end_dt)
select term_cls,
       sum(indc_qty)                   as indc_qty,
       sum(ac_stor_qty_kor)            as ac_stor_qty_kor,
       sum(sale_qty_kor)               as sale_qty_kor,
       sum(ac_sale_qty_kor)            as ac_sale_qty_kor,
       sum(stock_qty)                  as stock_qty,
       sum(ac_stor_qty_kor_season_end) as ac_stor_qty_kor_season_end,
       sum(ac_sale_qty_kor_season_end) as ac_sale_qty_kor_season_end,
       sum(stock_qty_season_end)       as stock_qty_season_end
from (
         select term_cls
              , indc_qty
              , 0 as ac_stor_qty_kor
              , 0 as sale_qty_kor
              , 0 as ac_sale_qty_kor
              , 0 as stock_qty
              , 0 as ac_stor_qty_kor_season_end
              , 0 as ac_sale_qty_kor_season_end
              , 0 as stock_qty_season_end
         from order_status
         union all
         select term_cls
              , 0 as indc_qty
              , ac_stor_qty_kor
              , sale_qty_kor
              , ac_sale_qty_kor
              , stock_qty
              , 0 as ac_stor_qty_kor_season_end
              , 0 as ac_sale_qty_kor_season_end
              , 0 as stock_qty_season_end
         from sale
         union all
         select term_cls
              , 0 as indc_qty
              , 0 as ac_stor_qty_kor
              , 0 as sale_qty_kor
              , 0 as ac_sale_qty_kor
              , 0 as stock_qty
              , ac_stor_qty_kor_season_end
              , ac_sale_qty_kor_season_end
              , stock_qty_season_end
         from season_end
     ) a
group by term_cls
order by term_cls

        """.format(
            para_brand=brand,
            para_season=season,
            para_season_py=season_py,
            para_season_py2=season_py2,
            para_category=category,
            para_sub_category=sub_category,
            para_adult_kids=adult_kids,
            para_start_dt=start_dt,
            para_end_dt=end_dt,
            para_end_dt_this_week=end_dt_this_week,
        )

In [7]:
cursor.execute(query)

In [9]:
cursor.execute("ROLLBACK")
connect.commit()

In [8]:
query_output = cursor.fetchall()

In [9]:
query_output

[('당해', 222792, 214035, 7257, 67134, 152673, 0, 0, 0),
 ('전년', 244505, 133471, 11616, 63499, 73050, 251980, 185803, 53699)]

In [10]:
from utils.redshift_data import RedshiftData

In [11]:
import pandas.io.sql as psql

In [12]:
redshift_data = RedshiftData(connect, query)

In [13]:
data = redshift_data.get_data()

/opt/homebrew/Caskroom/miniconda/base/envs/fnf/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
data

,term_cls,indc_qty,ac_stor_qty_kor,sale_qty_kor,ac_sale_qty_kor,stock_qty,ac_stor_qty_kor_season_end,ac_sale_qty_kor_season_end,stock_qty_season_end
0,당해,222792,214035,7257,67134,152673,0,0,0
1,전년,244505,133471,11616,63499,73050,251980,185803,53699


In [15]:
data["ac_sale_qty_kor"]

0    67134
1    63499
Name: ac_sale_qty_kor, dtype: int64

In [20]:
data["ac_stor_qty_kor"]

0    214035
1    133471
Name: ac_stor_qty_kor, dtype: int64

In [21]:
67134/214035 * 100

31.365898100777912

In [22]:
data

,term_cls,indc_qty,ac_stor_qty_kor,sale_qty_kor,ac_sale_qty_kor,stock_qty,ac_stor_qty_kor_season_end,ac_sale_qty_kor_season_end,stock_qty_season_end
0,당해,222792,214035,7257,67134,152673,0,0,0
1,전년,244505,133471,11616,63499,73050,251980,185803,53699


In [49]:
data['sales_rate'] = round((data['ac_sale_qty_kor']/data['ac_stor_qty_kor'])*100,0)

In [50]:
data["season_end_sales_rate"] = round((data["ac_sale_qty_kor_season_end"]/data["ac_stor_qty_kor_season_end"])*100,0)

In [51]:
data

,term_cls,indc_qty,ac_stor_qty_kor,sale_qty_kor,ac_sale_qty_kor,stock_qty,ac_stor_qty_kor_season_end,ac_sale_qty_kor_season_end,stock_qty_season_end,sales_rate,season_end_sales_rate
0,당해,222792,214035,7257,67134,152673,0,0,0,31.0,NaN
1,전년,244505,133471,11616,63499,73050,251980,185803,53699,48.0,74.0


In [52]:
data.fillna(0, inplace=True)

In [53]:
data

,term_cls,indc_qty,ac_stor_qty_kor,sale_qty_kor,ac_sale_qty_kor,stock_qty,ac_stor_qty_kor_season_end,ac_sale_qty_kor_season_end,stock_qty_season_end,sales_rate,season_end_sales_rate
0,당해,222792,214035,7257,67134,152673,0,0,0,31.0,0.0
1,전년,244505,133471,11616,63499,73050,251980,185803,53699,48.0,74.0
